# Setup

In [1]:
# Weights and Biases
!pip install -q wandb
# Tensorflow
!pip install -q tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00


In [2]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.math import multiply
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError
from math import log

In [3]:
# If running in colab, insert your wandb key here

#import config
#Erlend
#wandb.login(key=config.erlend_key)
# Hjalmar
wandb.login(key="b47bcf387a0571c5520c58a13be35cda8ada0a99")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Load, split and normalize data

### Load data

In [4]:
google_colab = True

if google_colab:
    import tensorflow as tf
    # Pring info
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

    # Code to read csv file into Colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "1Ic73MqpS5ACG1p2oJ_R5cTuEhlQRM_Q6"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('2013-2022_wo_lags.csv')  
    df_read = pd.read_csv('2013-2022_wo_lags.csv')
else:
    file = "../data/processed_data/2020_2022_moneyness_filtere.csv"
    df_read = pd.read_csv(file)

display(df_read)

Sat Apr 15 14:58:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

,Unnamed: 0,Quote_date,Price,Underlying_last,Strike,TTM,R
0,0,2013-01-02,361.355,1462.33,1100.0,2,0.07
1,1,2013-01-02,336.600,1462.33,1125.0,2,0.07
2,2,2013-01-02,311.350,1462.33,1150.0,2,0.07
3,3,2013-01-02,286.345,1462.33,1175.0,2,0.07
4,4,2013-01-02,281.345,1462.33,1180.0,2,0.07
...,...,...,...,...,...,...,...
12459173,12459173,2022-12-30,6.950,3839.81,8400.0,1085,4.22
12459174,12459174,2022-12-30,5.950,3839.81,8600.0,1085,4.22
12459175,12459175,2022-12-30,5.450,3839.81,8800.0,1085,4.22
12459176,12459176,2022-12-30,4.500,3839.81,9000.0,1085,4.22


In [5]:
df = df_read
del df_read

# Group the data by Quote Date and calculate the mean for Underlying Price
df_agg = df.groupby('Quote_date').mean().reset_index()

# Values to returns
df_agg["Underlying_return"] = df_agg["Underlying_last"].pct_change()

lags = 10

# Add the Underlying Price Lag column
for i in range(1, lags + 1):
    df_agg['Underlying_' + str(i)] = df_agg['Underlying_return'].shift(i)

df = pd.merge(df, df_agg[['Quote_date', "Underlying_return"] + ['Underlying_' + str(i) for i in range(1, lags + 1)]], on='Quote_date', how='left')

# Filter df between 2014-01-01 and 2022-12-31
df = df[(df["Quote_date"] >= "2014-01-01") & (df["Quote_date"] <= "2022-12-31")]

In [6]:
display(df)

,Unnamed: 0,Quote_date,Price,Underlying_last,Strike,TTM,R,Underlying_return,Underlying_1,Underlying_2,Underlying_3,Underlying_4,Underlying_5,Underlying_6,Underlying_7,Underlying_8,Underlying_9,Underlying_10
405271,405271,2014-01-03,555.455,1831.47,1275.0,7,0.02,-0.008816,0.003677,-0.000147,-0.000331,0.004664,0.002828,0.004721,0.005565,-0.000508,0.016502,-0.003006
405272,405272,2014-01-03,530.500,1831.47,1300.0,7,0.02,-0.008816,0.003677,-0.000147,-0.000331,0.004664,0.002828,0.004721,0.005565,-0.000508,0.016502,-0.003006
405273,405273,2014-01-03,505.495,1831.47,1325.0,7,0.02,-0.008816,0.003677,-0.000147,-0.000331,0.004664,0.002828,0.004721,0.005565,-0.000508,0.016502,-0.003006
405274,405274,2014-01-03,480.355,1831.47,1350.0,7,0.02,-0.008816,0.003677,-0.000147,-0.000331,0.004664,0.002828,0.004721,0.005565,-0.000508,0.016502,-0.003006
405275,405275,2014-01-03,455.350,1831.47,1375.0,7,0.02,-0.008816,0.003677,-0.000147,-0.000331,0.004664,0.002828,0.004721,0.005565,-0.000508,0.016502,-0.003006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12459173,12459173,2022-12-30,6.950,3839.81,8400.0,1085,4.22,-0.002300,0.017343,-0.012034,-0.004081,0.000096,0.005621,-0.014208,0.014673,0.001048,-0.020257,-0.024582
12459174,12459174,2022-12-30,5.950,3839.81,8600.0,1085,4.22,-0.002300,0.017343,-0.012034,-0.004081,0.000096,0.005621,-0.014208,0.014673,0.001048,-0.020257,-0.024582
12459175,12459175,2022-12-30,5.450,3839.81,8800.0,1085,4.22,-0.002300,0.017343,-0.012034,-0.004081,0.000096,0.005621,-0.014208,0.014673,0.001048,-0.020257,-0.024582
12459176,12459176,2022-12-30,4.500,3839.81,9000.0,1085,4.22,-0.002300,0.017343,-0.012034,-0.004081,0.000096,0.005621,-0.014208,0.014673,0.001048,-0.020257,-0.024582


### Format input data

In [7]:
# Format settings
max_timesteps = lags
moneyness = False
bs_vars = ['Moneyness', 'TTM', 'R'] if moneyness else ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = ['Underlying_last'] + [f'Underlying_{i}' for i in range (1, max_timesteps)]

def create_rw_dataset(window_number = 0, df = df):
    '''Creates dataset for a single rolling window period offsett by the window number'''

    # Create train, validation and test set split points
    test_weeks = 2
    train_start = datetime(2014,9,23) + relativedelta(weeks=window_number * test_weeks)
    val_start = train_start + relativedelta(months=3)
    test_start = val_start + relativedelta(weeks=1)
    test_end = test_start + relativedelta(weeks=test_weeks)
    train_start = str(train_start.date())
    val_start = str(val_start.date())
    test_start = str(test_start.date())
    test_end = str(test_end.date())

    # Add moneyness column
    if not moneyness:
        df['Moneyness'] = df['Underlying_last'] / df['Strike']


    # Split train and validation data
    df_train = df[(df['Quote_date'] >= train_start) & (df['Quote_date'] < val_start)]
    df_val = df[(df['Quote_date'] >= val_start) & (df['Quote_date'] < test_start)]
    df_test = df[(df['Quote_date'] >= test_start) & (df['Quote_date'] < test_end)]

    del df

    # Filter out top and bottom 5% of moneyness in df_train, and using the same filter in df_val and df_test
    top = df_train['Moneyness'].quantile(0.95)
    bottom = df_train['Moneyness'].quantile(0.05)
    df_train = df_train[(df_train['Moneyness'] <= top) & (df_train['Moneyness'] >= bottom)]
    df_val = df_val[(df_val['Moneyness'] <= top) & (df_val['Moneyness'] >= bottom)]
    df_test = df_test[(df_test['Moneyness'] <= top) & (df_test['Moneyness'] >= bottom)]


    # Extract target values
    train_y = (df_train['Price'] / df_train['Strike']).to_numpy() if moneyness else df_train['Price'].to_numpy()
    val_y = (df_val['Price'] / df_val['Strike']).to_numpy() if moneyness else df_val['Price'].to_numpy()
    test_y = (df_test['Price'] / df_test['Strike']).to_numpy() if moneyness else df_test['Price'].to_numpy()

    # If usining moneyness, extract strike
    if moneyness:
        train_strike = df_train['Strike'].to_numpy()
        val_strike = df_val['Strike'].to_numpy()
        test_strike = df_test['Strike'].to_numpy()

    # Convert dataframes to numpy arrays
    train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
    val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]
    test_x = [df_test[underlying_lags].to_numpy(), df_test[bs_vars].to_numpy()]

    del df_train
    del df_val

    # Scale features based on training set
    underlying_scaler = MinMaxScaler()
    train_x[0] = underlying_scaler.fit_transform(train_x[0])
    val_x[0] = underlying_scaler.transform(val_x[0])
    test_x[0] = underlying_scaler.transform(test_x[0])

    bs_scaler = MinMaxScaler()
    train_x[1] = bs_scaler.fit_transform(train_x[1])
    val_x[1] = bs_scaler.transform(val_x[1])
    test_x[1] = bs_scaler.transform(test_x[1])


    # Shuffle training set
    np.random.seed(0)
    shuffle = np.random.permutation(len(train_x[0]))
    train_x = [train_x[0][shuffle], train_x[1][shuffle]]
    train_y = train_y[shuffle]
    if moneyness:
        train_strike = train_strike[shuffle]

    # Reshape data to fit LSTM
    train_x = [train_x[0].reshape(len(train_x[0]), max_timesteps, 1), train_x[1]]
    val_x = [val_x[0].reshape(len(val_x[0]), max_timesteps, 1), val_x[1]]
    test_x = [test_x[0].reshape(len(test_x[0]), max_timesteps, 1), test_x[1]]

    print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
    print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')
    print(f'Test shape: {test_x[0].shape}, {test_x[1].shape}')

    if moneyness:
        return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike,
    return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test

# Create the dataset for the first rolling window period
if moneyness:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike = create_rw_dataset()
else:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset()

Train shape: (172963, 10, 1), (172963, 4)
Val shape: (11980, 10, 1), (11980, 4)
Test shape: (25322, 10, 1), (25322, 4)


# Model construction

In [8]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.Interface_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), model(underlying_history), model(underlying_history), model(underlying_history), model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Hyperparameter search setup

In [9]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'bayes',
    'name': 'LSTM-MLP v4.0: fix nan issue',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [4, 8, 16, 32]},
        'Interface_units': {
            'values': [4, 8, 16, 32]},
        'MLP_units': {
            'values': [50, 100, 200, 400, 600]},
        'LSTM_timesteps': {
            'values': [10, 20, 40, 60, 90, 150]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'Bn_momentum': {
            'values': [0.1, 0.4, 0.7, 0.99]},
        'Lr': {
            'distribution': 'log_uniform',
            'max': log(0.1), 'min': log(0.0001)},
        'Lr_decay': {
            'distribution': 'log_uniform',
            'max': log(1), 'min': log(0.8)},        
        'Minibatch_size': {
            'value': 4096},
        'Min_delta': {
            'value': 0.01 if moneyness else 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 3 if moneyness else 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - test area') 
#sweep_id = '98bxt6oq'

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.
wandb: WARNING   Violation 2. Lr_decay uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: i0m10kyl
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing%20-%20test%20area/sweeps/i0m10kyl


# Run hyperparameter search

In [10]:
#WIP
class MSE_LossCallback(Callback):
    def __init__(self, train_x, train_y, train_strike, val_x, val_y, val_strike):
        self.train_x = train_x
        self.train_y = train_y
        self.train_strike = train_strike
        self.val_x = val_x
        self.val_y = val_y
        self.val_strike = val_strike
    
    def on_epoch_end(self, epoch, logs={}):
        train_pred = self.model(train_x)
        val_pred = self.model(val_x)

        train_mse = reduce_mean(square(multiply(train_pred[:,0] - self.train_y, self.train_strike)))
        val_mse = reduce_mean(square(multiply(val_pred[:,0] - self.val_y, self.val_strike)))

        print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')


In [11]:
# Calculate the training and validation MSE loss on the actual option price when using price/strike as the target
def MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike):
    train_pred = model(train_x)
    val_pred = model(val_x)

    train_mse = reduce_mean(square((train_pred[:,0] - train_y)*train_strike))
    val_mse = reduce_mean(square((val_pred[:,0] - val_y)*val_strike))

    print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')

In [12]:
import gc
from tensorflow.keras import backend as k

class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

## Creating trainer function

In [13]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None, previous_checkpoint_path = None, checkpoint_path = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        if previous_checkpoint_path and os.path.exists(previous_checkpoint_path + ".h5"):
            model = load_model(previous_checkpoint_path + ".h5")
        else:
            model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )
        
        # Check if the checkpoint folder exists
        if checkpoint_path and not os.path.exists(checkpoint_path):
            # Create the checkpoint folder if it does not exist
            os.makedirs(checkpoint_path)
        
        checkpoint = ModelCheckpoint(
            filepath=checkpoint_path + ".h5",
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            save_weights_only=False
        )

        # Adapt sequence length to config
        train_x_adjusted = [train_x[0][:, :config.LSTM_timesteps, :], train_x[1]]
        val_x_adjusted = [val_x[0][:, :config.LSTM_timesteps, :], val_x[1]]
        print(f'Train shape: {train_x_adjusted[0].shape}, {train_x_adjusted[0].shape}')
        print(f'Val shape: {val_x_adjusted[0].shape}, {val_x_adjusted[0].shape}')

        # Train model
        model.fit(
            train_x_adjusted,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x_adjusted, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback, checkpoint, ClearMemory()] if checkpoint_path else [early_stopping, wandb_callback, ClearMemory()],
        )

        if moneyness:
            MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike)

### Run full sweep

In [14]:
#wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 100)

### Single run

# Rolling window

In [15]:
def calculate_error(predictions, original):
    m = MeanSquaredError()
    m.update_state(predictions, original)
    print("MSE:", m.result().numpy())
    m = RootMeanSquaredError()
    m.update_state(predictions, original)
    print("RMSE:", m.result().numpy())

class config_object:
    def __init__(self, config):
        self.LSTM_units = config['LSTM_units']
        self.Interface_units = config['Interface_units']
        self.MLP_units = config['MLP_units']
        self.LSTM_timesteps = config['LSTM_timesteps']
        self.LSTM_layers = config['LSTM_layers']
        self.MLP_layers = config['MLP_layers']
        self.Bn_momentum = config['Bn_momentum']
        self.Lr = config['Lr']
        self.Lr_decay = config['Lr_decay']
        self.Minibatch_size = config['Minibatch_size']
        self.Min_delta = config['Min_delta']
        self.Patience = config['Patience']
        self.Num_features = config['Num_features']
        self.Architecture = config['Architecture']
        

In [ ]:
num_windows = 26 #84

config = {
    'LSTM_units': 4,
    'Interface_units': 16,
    'MLP_units': 600,
    'LSTM_timesteps': 10,
    'LSTM_layers': 6,
    'MLP_layers': 7,
    'Bn_momentum': 0.99,
    'Lr': 0.005,
    'Lr_decay': 0.89,
    'Minibatch_size': 4096,
    'Min_delta': 0.01 if moneyness else 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4, 
    'Architecture': 'LSTM-MLP v.5.0',
}

# Ask before training, so that you don't have to verify later
from google.colab import drive
if google_colab == True:
  drive.mount('/content/drive')

df_test_combined = pd.DataFrame()

checkpoint_time = datetime.now().strftime("%m-%d_%H-%M")

previous_checkpoint_path = None

for window in range(num_windows):
    if moneyness:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike, = create_rw_dataset(window)
    else:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(window)

    checkpoint_path = f'/content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/{checkpoint_time}/{train_start}/'

    print('-------------------------------------------')
    print("Checkpoint_path = ", checkpoint_path)
    print("Previous checkpoint_path = ", previous_checkpoint_path)
    print("Test start: ", test_start)
    print('-------------------------------------------')

    config['Dataset'] = f'{train_start} - {val_start} - {test_start}'

    trainer(config = config, project = 'Deep learning for option pricing - rolling windows', previous_checkpoint_path = previous_checkpoint_path, checkpoint_path = checkpoint_path)
    c_model = load_model(checkpoint_path + ".h5")
    predictions = np.array(c_model(test_x))
    print(f'--- Predictions for test_start {test_start} ---')
    calculate_error(predictions, test_y)
    print('-------------------------------------------')
    df_test["Prediction"] = predictions
    df_test_combined = pd.concat([df_test_combined, df_test[["Quote_date", "Price", "Prediction"] + bs_vars]])
    
    previous_checkpoint_path = checkpoint_path


print(f"--- All model predictions ---")
calculate_error(df_test_combined["Prediction"], df_test_combined["Price"])
print("-------------------------------------------")

if google_colab == False:
    predictions_path = './predictions/'
    if checkpoint_path and not os.path.exists(predictions_path):
        os.makedirs(predictions_path)
    df_test_combined.to_csv(f'{predictions_path}{datetime.now().strftime("%m-%d_%H-%M")}.csv')

if google_colab == True:
  path = '/content/drive/My Drive/Predictions/predictions_2015_v5_4_2weektest_modelload.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    df_test_combined.to_csv(f)

Mounted at /content/drive
Train shape: (172963, 10, 1), (172963, 4)
Val shape: (11980, 10, 1), (11980, 4)
Test shape: (25322, 10, 1), (25322, 4)
-------------------------------------------
Checkpoint_path =  /content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/04-15_14-59/2014-09-23/
Previous checkpoint_path =  None
-------------------------------------------


wandb: Currently logged in as: vinje (avogadro). Use `wandb login --relogin` to force relogin


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 16)           2016        ['input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]']                
                                                                                              

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▃▃▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,33
best_val_loss,15.44719
epoch,53
loss,34.90315
val_loss,30.24173


--- Predictions for test_start 2014-12-30 ---
MSE: 29.728848
RMSE: 5.452417
-------------------------------------------
Train shape: (173097, 10, 1), (173097, 4)
Val shape: (14083, 10, 1), (14083, 4)
Test shape: (24916, 10, 1), (24916, 4)
-------------------------------------------
Checkpoint_path =  /content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/04-15_14-59/2014-10-07/
Previous checkpoint_path =  /content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/04-15_14-59/2014-09-23/
-------------------------------------------


...

--- Predictions for test_start 2015-02-11 ---
MSE: 497.70966
RMSE: 22.309406
-------------------------------------------
Train shape: (172749, 10, 1), (172749, 4)
Val shape: (14949, 10, 1), (14949, 4)
Test shape: (32907, 10, 1), (32907, 4)
-------------------------------------------
Checkpoint_path =  /content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/04-15_14-59/2014-11-18/
Previous checkpoint_path =  /content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/04-15_14-59/2014-11-04/
-------------------------------------------


Train shape: (172963, 10, 1), (172963, 10, 1)
Val shape: (11980, 10, 1), (11980, 10, 1)
Epoch 1/1000
43/43 [==============================] - 36s 274ms/step - loss: 31.6365 - val_loss: 28.1635
Epoch 2/1000
43/43 [==============================] - 10s 105ms/step - loss: 32.9428 - val_loss: 34.4918
Epoch 3/1000
43/43 [==============================] - 10s 105ms/step - loss: 36.8697 - val_loss: 30.9200
Epoch 4/1000
43/43 [==============================] - 10s 106ms/step - loss: 32.2366 - val_loss: 29.4026
Epoch 5/1000
43/43 [==============================] - 10s 108ms/step - loss: 39.8920 - val_loss: 30.6359
Epoch 6/1000
43/43 [==============================] - 10s 105ms/step - loss: 42.2519 - val_loss: 30.0585
Epoch 7/1000
43/43 [==============================] - 10s 107ms/step - loss: 34.3446 - val_loss: 28.5589
Epoch 8/1000
43/43 [==============================] - 9s 106ms/step - loss: 38.6673 - val_loss: 28.1695
Epoch 9/1000
43/43 [==============================] - 10s 130ms/step - lo

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▂▂▃▂▅▅▃▄▃▄▃▂▇▄▆▂▁▄▆▁▂▅▃▄▅▄█▄▂▄▄▆▃▁▃▅▂▃▂▂
val_loss,▃█▅▄▅▅▃▃▃▂▃▂▃▃▃▃▂▁▂▃▂▁▁▁▂▁▂▂▂▁▁▁▂▃▂▁▂▁▂▂
best_epoch,24
best_val_loss,25.57425
epoch,44
loss,33.75679
val_loss,26.93236


--- Predictions for test_start 2015-02-25 ---
MSE: 370.88254
RMSE: 19.25831
-------------------------------------------
Train shape: (171868, 10, 1), (171868, 4)
Val shape: (16649, 10, 1), (16649, 4)
Test shape: (34131, 10, 1), (34131, 4)
-------------------------------------------
Checkpoint_path =  /content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/04-15_14-59/2014-12-02/
Previous checkpoint_path =  /content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/04-15_14-59/2014-11-18/
-------------------------------------------


Train shape: (172963, 10, 1), (172963, 10, 1)
Val shape: (11980, 10, 1), (11980, 10, 1)
Epoch 1/1000


### Load single model

In [ ]:
config = {
    'LSTM_units': 4,
    'Interface_units': 4,
    'MLP_units': 600,
    'LSTM_timesteps': 10,
    'LSTM_layers': 3,
    'MLP_layers': 7,
    'Bn_momentum': 0.99,
    'Lr': 0.0001,
    'Lr_decay': 0.92,
    'Minibatch_size': 4096,
    'Min_delta': 0.01 if moneyness else 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4, 
    'Architecture': 'LSTM-MLP v.1.0',
}

window = 1
if moneyness:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike, = create_rw_dataset(window)
else:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(window)

checkpoint_path = f'./checkpoint/03-20_12-35/{train_start}/'

co = config_object(config)
c_model = create_model(co)
c_model.load_weights(checkpoint_path)
predictions = np.array(c_model(test_x))
print(f'--- Predictions for {test_start} ---')
calculate_error(predictions, test_y)
print('-------------------------------------------')